In [1]:
%load_ext autoreload
%autoreload 2

# Visualise ruggedness verified for trained models

In [ ]:
from synbio_morpher.utils.data.data_format_tools.common import load_json_as_dict
from synbio_morpher.utils.results.analytics.naming import get_true_interaction_cols
from synbio_morpher.utils.results.analytics.timeseries import calculate_adaptation
from evoscaper.utils.dataset import embellish_data
from evoscaper.model.evaluation import calc_prompt_adherence
from evoscaper.scripts.init_from_hpos import init_from_hpos
from evoscaper.scripts.verify import verify
from evoscaper.utils.math import arrayise
from evoscaper.utils.preprocess import make_datetime_str
from evoscaper.utils.evolution import calculate_ruggedness_core

from functools import partial
import os
import json
import sys
import numpy as np
import jax
import jax.numpy as jnp

import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

jax.config.update('jax_platform_name', 'cpu')


# if __package__ is None:

module_path = os.path.abspath(os.path.join('..'))
sys.path.append(module_path)

__package__ = os.path.basename(module_path)


np.random.seed(0)
PRNG = jax.random.PRNGKey(0)

jax.devices()

# jupyter nbconvert --to notebook --execute 02_cvae_verify.ipynb --output=02_cvae_verify_2.ipynb --ExecutePreprocessor.timeout=-1



xla_bridge.py:backends():900: Unable to initialize backend 'rocm': NOT_FOUND: Could not find registered platform with name: "rocm". Available platform names are: CUDA INFO
xla_bridge.py:backends():900: Unable to initialize backend 'tpu': INTERNAL: Failed to open libtpu.so: libtpu.so: cannot open shared object file: No such file or directory INFO


[CpuDevice(id=0)]

# Settings + load params

In [3]:
dir_src = os.path.join('data', '01_cvae', '2025_03_16__17_28_06')
hpos = pd.Series(load_json_as_dict(os.path.join(dir_src, 'hpos_all.json')))
fn_saves = os.path.join(dir_src, 'saves_2025_03_16__17_28_06_ds0129_a_hs32_nl3_KL2e4_cont01ts095pd3_lr1e3_teva97_eqcLogNoneg')
dir_src_rugg = os.path.join('data', 'ruggedness', '2025_03_14__16_42_13')

cfg = {'fn_saves': fn_saves,
       'dir_src_rugg': dir_src_rugg}
top_write_dir = os.path.join('data', '02_cvae_verify', make_datetime_str())
os.makedirs(top_write_dir, exist_ok=True)
with open(os.path.join(top_write_dir, 'cfg.json'), 'w') as f:
    json.dump(cfg, f)
print('top_write_dir:', top_write_dir)
    

top_write_dir: data/02_cvae_verify/2025_03_18__14_47_35


In [ ]:
saves_loaded = load_json_as_dict(fn_saves)
(
    rng, rng_model, rng_dataset,
    config_norm_x, config_norm_y, config_filter, config_optimisation, config_dataset, config_training, config_model,
    data, x_cols, df,
    x, cond, y, x_train, cond_train, y_train, x_val, cond_val, y_val,
    total_ds, n_batches, BATCH_SIZE, x_datanormaliser, x_methods_preprocessing, y_datanormaliser, y_methods_preprocessing,
    params, encoder, decoder, model, h2mu, h2logvar, reparam
) = init_from_hpos(hpos)

params = saves_loaded[str(list(saves_loaded.keys())[-1])]['params']
params = arrayise(params)

config_bio = load_json_as_dict(config_dataset.filenames_verify_config)
if 'base_configs_ensemble' in config_bio:
    val_config = config_bio['base_configs_ensemble']['generate_species_templates']
    val_config.update(config_bio['base_configs_ensemble']['mutation_effect_on_interactions_signal'])
else:
    val_config = config_bio
    
analytics = load_json_as_dict(os.path.join(dir_src_rugg, 'analytics.json'))
config_rugg = load_json_as_dict(os.path.join(dir_src_rugg, 'config.json'))

for k, v in analytics.items():
    analytics[k] = np.array(v)
    

/usr/local/lib/python3.10/dist-packages/pandas/core/arraylike.py:399: RuntimeWarning: divide by zero encountered in log10
  result = getattr(ufunc, method)(*inputs, **kwargs)


In [ ]:
from evoscaper.utils.simulation import load_config_bio, sim, prep_cfg, setup_model_brn, compute_analytics
from evoscaper.scripts.cvae_scan import generate_all_fake_circuits

df_hpos = pd.read_json(os.path.join(dir_src, 'hpos_all.json'))
df_hpos['eval_n_to_sample'] = 14284 # config_rugg['eval_n_to_sample']
df_hpos['eval_cond_min'] = config_rugg['eval_cond_min']
df_hpos['eval_cond_max'] = config_rugg['eval_cond_max']
df_hpos['eval_n_categories'] = config_rugg['eval_n_categories']

datasets = {v: pd.read_json(
    v) for v in df_hpos['filenames_train_table'].unique() if os.path.exists(v)}
input_species = datasets[list(datasets.keys())[
    0]]['sample_name'].dropna().unique()
fn_config_bio = df_hpos['filenames_verify_config'].dropna().unique()[0]

config_bio = load_config_bio(
    fn_config_bio, input_species, '../' + config_rugg.get('fn_simulation_settings'))

model_brn, qreactions, postprocs, ordered_species = setup_model_brn(
    config_bio, input_species)

df_hpos['output_species'] = df_hpos['output_species'].apply(lambda x: [x])

all_fake_circuits, all_forward_rates, all_reverse_rates, all_sampled_cond = generate_all_fake_circuits(
    df_hpos, datasets, input_species, postprocs)

In [24]:
analytics['adaptation'] = calculate_adaptation(analytics['sensitivity'], analytics['precision'])

In [ ]:
analytics_1 = load_json_as_dict(os.path.join(dir_src, 'batch_1', 'analytics.json'))

14284.0

In [28]:
df_hpos['eval_n_to_sample'] = 14284 # config_rugg['eval_n_to_sample']
all_fake_circuits, all_forward_rates, all_reverse_rates, all_sampled_cond = generate_all_fake_circuits(
    df_hpos, datasets, input_species, postprocs)

n_samples = all_fake_circuits.shape[0]
n_interactions = all_fake_circuits.shape[1]
n_perturbs = n_interactions + config_rugg['resimulate_analytics']
eps = config_rugg['eps_perc'] * np.abs(all_fake_circuits).max()

for analytic in config_dataset.objective_col:
    ruggedness = calculate_ruggedness_core(analytics, None, analytic,
                                       config_rugg['resimulate_analytics'], n_samples, n_perturbs, eps)
    break

/usr/local/lib/python3.10/dist-packages/pandas/core/arraylike.py:399: RuntimeWarning: divide by zero encountered in log10
  result = getattr(ufunc, method)(*inputs, **kwargs)


TypeError: cannot reshape array of shape (99988, 9) (size 899892) into shape (14280, 4, -1) because the product of specified axis sizes (57120) does not evenly divide 899892